In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import os
from torch import optim
from torchvision import models
import numpy as np

In [26]:
class SiameseNet(nn.Module):
    def __init__(self, hidden_size = [256, 100]):
        super(SiameseNet, self).__init__()
        self.hidden_size = hidden_size
        
        # ResNet requires the input to be the size of [batch_size, 3, 224, 224]
        self.ResNet34 = models.resnet34(pretrained=True)
        for param in self.ResNet34.parameters():
            param.requires_grad = False
        self.ResNet34.fc = nn.Linear(512, hidden_size[0])
        self.ResNet34.fc.requires_grad = True
        
        self.FC1 = nn.Linear(2 * hidden_size[0], hidden_size[1])
        self.Output = nn.Linear(hidden_size[1], 1)
    
    def forward(self, img1, img2):
        out1 = self.ResNet34(img1).view(-1, self.hidden_size[0])
        out2 = self.ResNet34(img2).view(-1, self.hidden_size[0])
        out = torch.cat([out1, out2], dim=1)
        out = F.tanh(self.FC1(out))
        out = self.Output(out)
        return out

In [47]:
x1 = Variable(torch.rand(32, 3, 224, 224)).cuda()
x2 = Variable(torch.rand(32, 3, 224, 224)).cuda()
diff = Variable(torch.rand(32, 1)).cuda()

In [48]:
batch_size, num_classes = 32, 3
label_type = None

model = SiameseNet().cuda()
# y_pred = model.forward(x1, x2)
criterion = nn.MSELoss()
if label_type and label_type == 'categorical':
    criterion == nn.CrossEntropyLoss()
params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(params, lr=0.0001)

losses = []

In [42]:
def mse_loss(input, target):
    return torch.sum((input - target) ** 2) / input.data.nelement()

In [49]:
for epoch in range(500):
    # Forward pass
    diff_pred = model.forward(x1, x2)
    
    # compute and print loss
    loss = mse_loss(diff_pred, diff)
    losses.append(loss.data)
    if epoch % 100 == 0:
        print(epoch, loss.data)
        
    # zero grads
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 tensor(0.3189, device='cuda:0')
100 tensor(1.00000e-06 *
       8.9875, device='cuda:0')
200 tensor(1.00000e-10 *
       2.1463, device='cuda:0')
300 tensor(1.00000e-14 *
       1.5011, device='cuda:0')
400 tensor(1.00000e-15 *
       3.5024, device='cuda:0')
